In [1]:
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import matplotlib.pyplot as plt
import numpy as np
import io, unicodedata, re, string
import pandas as pd
df = pd.read_csv('train.csv')

In [2]:
map_wrong_words = {
    'fucksex': 'fuck sex',
    'yourselfgo': 'your self ego',
    'bcoz': 'because',
    'bc': 'because',
    'deneid': 'denied',
    '\u200e': '',
    'wriminalwar': 'criminal war',
    'Pathectic': 'pathetic',
    'POLITCAL': 'political',
    'talk2me': 'talk to me',
    'shitfuck': 'shift fuck',
    'BabyWhat': 'baby what',
    'Sockpuppetry': 'sock puppetry',
    'Bastered': 'bastard',
    'PHILIPPINESLONG': 'philippines long',
    'SuPeRTR0LL': 'supertroll',
    'FUCKBAGS': 'fuck bags',
    'peNis': 'penis',
    'pensnsnnienSNsn': 'penis',
    'pneis': 'penis', 
    'FooL': 'fool',
    'pennnis': 'penis',
    'PenIS': 'penis',
    'itsuck': 'it suck',
    'deletionist': 'delete',
    'ReSPeCT': 'respect',
    'stuipd' : 'stupid',
    '@sshole': 'asshole',
    'mf': 'mother fucker',
    'b@stard': 'bastard',
    's0n': 'son',
    'b!tch': 'bitch'
}

def clean_wrong_spell_words(text, mapping):
    for word in mapping:
        text = text.replace(word, mapping[word])
    return text
df['comment_text'] = df['comment_text'].apply(lambda x : clean_wrong_spell_words(x,map_wrong_words))


In [3]:
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext

def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

df['comment_text'] = df['comment_text'].apply(lambda x: cleanHtml(x))
df['comment_text'] = df['comment_text'].apply(lambda x: cleanPunc(x))
df['comment_text'] = df['comment_text'].apply(lambda x: keepAlpha(x))

In [4]:
def remove_special(cleaned):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',cleaned)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    cleaned = cleaned.lower()
    return cleaned
df['comment_text'] = df['comment_text'].apply(lambda x: remove_special(x))


In [5]:
contraction_mapping = {
    "ain't": "is not", 
    "aren't": "are not",
    "can't": "cannot", 
    "'cause": "because", 
    "could've": "could have", 
    "couldn't": "could not", 
    "didn't": "did not",  
    "doesn't": "does not", 
    "don't": "do not", 
    "hadn't": "had not", 
    "hasn't": "has not", 
    "haven't": "have not", 
    "he'd": "he would",
    "he'll": "he will", 
    "he's": "he is", 
    "how'd": "how did", 
    "how'd'y": "how do you", 
    "how'll": "how will", 
    "how's": "how is",  
    "I'd": "I would", 
    "I'd've": "I would have", 
    "I'll": "I will", 
    "I'll've": "I will have",
    "I'm": "I am", 
    "I've": "I have", 
    "i'd": "i would", 
    "i'd've": "i would have", 
    "i'll": "i will",  
    "i'll've": "i will have",
    "i'm": "i am", 
    "i've": "i have", 
    "isn't": "is not", 
    "it'd": "it would", 
    "it'd've": "it would have", 
    "it'll": "it will", "it'll've": 
    "it will have","it's": "it is", 
    "let's": "let us", 
    "ma'am": "madam", 
    "mayn't": "may not", 
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have", 
    "must've": "must have", 
    "mustn't": "must not", 
    "mustn't've": "must not have", 
    "needn't": "need not", 
    "needn't've": "need not have",
    "o'clock": "of the clock", 
    "oughtn't": "ought not", 
    "oughtn't've": "ought not have", 
    "shan't": "shall not", 
    "sha'n't": "shall not", 
    "shan't've": "shall not have", 
    "she'd": "she would", 
    "she'd've": "she would have", 
    "she'll": "she will", 
    "she'll've": "she will have", 
    "she's": "she is", 
    "should've": "should have", 
    "shouldn't": "should not", 
    "shouldn't've": "should not have", 
    "so've": "so have",
    "so's": "so as", 
    "this's": "this is",
    "that'd": "that would", 
    "that'd've": "that would have", 
    "that's": "that is", 
    "there'd": "there would", 
    "there'd've": "there would have", 
    "there's": "there is", 
    "here's": "here is",
    "they'd": "they would", 
    "they'd've": "they would have", 
    "they'll": "they will", 
    "they'll've": "they will have", 
    "they're": "they are", 
    "they've": "they have", 
    "to've": "to have", 
    "wasn't": "was not", 
    "we'd": "we would", 
    "we'd've": "we would have", 
    "we'll": "we will", 
    "we'll've": "we will have", 
    "we're": "we are", 
    "we've": "we have", 
    "weren't": "were not", 
    "what'll": "what will", 
    "what'll've": "what will have", 
    "what're": "what are",  
    "what's": "what is", 
    "what've": "what have", 
    "when's": "when is", 
    "when've": "when have", 
    "where'd": "where did", 
    "where's": "where is", 
    "where've": "where have", 
    "who'll": "who will", 
    "who'll've": "who will have", 
    "who's": "who is", 
    "who've": "who have", 
    "why's": "why is", 
    "why've": "why have", 
    "will've": "will have", 
    "won't": "will not", 
    "won't've": "will not have", 
    "would've": "would have", 
    "wouldn't": "would not", 
    "wouldn't've": "would not have", 
    "y'all": "you all", 
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would", 
    "you'd've": "you would have", 
    "you'll": "you will", 
    "you'll've": "you will have", 
    "you're": "you are", 
    "you've": "you have" }
def remove_contractions(text, mapping):
    for word in mapping:
        text = text.replace(word, mapping[word])
    return text
df['comment_text'] = df['comment_text'].apply(lambda x: remove_contractions(x,contraction_mapping))


In [6]:
nltk.download('stopwords')
def remove_stopwords(text):
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    cleaned_text = ""
    for word in text.split():
        if word not in stop_words:
            cleaned_text += word + ' '
    return cleaned_text.strip() 
df['comment_text'] = df['comment_text'].apply(lambda x: remove_stopwords(x))
df['comment_text'].head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                  sir hero chance remember page thats
5             congratulations well use tools well talk
6                          cocksucker piss around work
7    vandalism matt shirvington article reverted pl...
8    sorry word nonsense offensive anyway im intend...
9                 alignment subject contrary dulithgow
Name: comment_text, dtype: object

In [7]:
df['comment_text'].head()

0    explanation edits made username hardcore metal...
1    daww matches background colour im seemingly st...
2    hey man im really trying edit war guy constant...
3    cant make real suggestions improvement wondere...
4                  sir hero chance remember page thats
Name: comment_text, dtype: object

In [8]:
x =  df.comment_text
y =  df.drop(['id','comment_text'],axis = 1)
print(x)

0         explanation edits made username hardcore metal...
1         daww matches background colour im seemingly st...
2         hey man im really trying edit war guy constant...
3         cant make real suggestions improvement wondere...
4                       sir hero chance remember page thats
                                ...                        
159566    second time asking view completely contradicts...
159567                 ashamed horrible thing put talk page
159568    spitzer umm theres actual article prostitution...
159569    looks like actually put speedy first version d...
159570    really dont think understand came idea bad rig...
Name: comment_text, Length: 159571, dtype: object


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test= train_test_split(x,y,test_size=0.25, random_state=1)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1,3), min_df=3, max_df=0.9, strip_accents='unicode', use_idf= True, smooth_idf=True, sublinear_tf= True)

In [11]:
transformer = vec.fit(x_train)
x_train_transformed = transformer.transform(x_train)
x_test_transformed = transformer.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score , accuracy_score , confusion_matrix , f1_score
from sklearn.multiclass import OneVsRestClassifier

logreg = LogisticRegression(C = 10, penalty='l2', solver = 'liblinear', random_state=0)

classifier = OneVsRestClassifier(logreg)
classifier.fit(x_train_transformed, y_train)


#y_train_pred_proba = classifier.predict_proba(x_train_transformed)
y_pred = classifier.predict_proba(x_test_transformed)


In [13]:
roc_auc_score_test = roc_auc_score(y_test, y_pred,average='weighted')
print(roc_auc_score_test)

0.9756306828605658


In [22]:
text=['Rakesh is an asshole']
text_transformed = transformer.transform(text)
categories = ['toxic','severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(categories)
print(classifier.predict(text_transformed))

text2=['you are a selfish piece of shit, you cock sucking motherfucker']
text2_transformed = transformer.transform(text2)
print(classifier.predict(text2_transformed))

text3=['i am very sorry to disturb you']
text3_transformed = transformer.transform(text3)
print(classifier.predict(text3_transformed))

text4=['Screw it in nice and slow']
text4_transformed = transformer.transform(text4)
ans=(classifier.predict(text4_transformed))
print(ans[0][0])

text5 = ['pass me the screw driver']
text5_transformed = transformer.transform(text5)
print(classifier.predict(text5_transformed))


['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
[[1 0 1 0 1 0]]
[[1 1 1 0 1 0]]
[[0 0 0 0 0 0]]
1
[[0 0 0 0 0 0]]


In [15]:
test = pd.read_csv('test.csv')
test['comment_text'] = test['comment_text'].apply(lambda x: clean_wrong_spell_words(x,map_wrong_words))
test['comment_text'] = test['comment_text'].apply(lambda x: cleanHtml(x))
test['comment_text'] = test['comment_text'].apply(lambda x: cleanPunc(x))
test['comment_text'] = test['comment_text'].apply(lambda x: keepAlpha(x))
test['comment_text'] = test['comment_text'].apply(lambda x: remove_special(x))
test['comment_text'] = test['comment_text'].apply(lambda x: remove_contractions(x,contraction_mapping))
test['comment_text'] = test['comment_text'].apply(lambda x: remove_stopwords(x))


In [16]:
test_x =  test.comment_text
test_x_transformed = transformer.transform(test_x)
test_y = classifier.predict_proba(test_x_transformed)

In [17]:
submission_df = pd.DataFrame({
    'id': test['id'],
    'toxic': test_y[:, 0],
    'severe_toxic': test_y[:, 1],
    'obscene': test_y[:, 2],
    'threat': test_y[:, 3],
    'insult': test_y[:, 4],
    'identity_hate': test_y[:, 5]
})
submission_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999925,0.118802,0.998282,0.027471,0.942335,0.180827
1,0000247867823ef7,0.000601,0.000802,0.000690,0.000773,0.003150,0.000689
2,00013b17ad220c46,0.011476,0.001125,0.004155,0.000429,0.006115,0.001350
3,00017563c3f7919a,0.003397,0.001736,0.001852,0.000376,0.001027,0.000226
4,00017695ad8997eb,0.017419,0.001599,0.002681,0.000794,0.011314,0.001510


In [18]:
submission_df.to_csv('submission.csv', index=False)